# Submission Generating(Test) code

In [ ]:
# For colab
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/Github/KT_Devchall"

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [ ]:
len(X_test)

In [ ]:
import random
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

set_config(display='diagram')

In [ ]:
# data load
X_test = pd.read_pickle('data/test_ID_treated.pkl')
X_test = X_test.sort_index() # bid index 순으로 정리
X_train = pd.read_pickle('data/train_ID_treated.pkl')

In [ ]:
## Column Selection
feature_importance = pd.Series([3.12768611e+01, 6.62209523e+00, 1.55948023e+01, 1.05818328e+00,
       5.35072461e-02, 0.00000000e+00, 3.21271373e+00, 2.57594949e-02,
       1.17493059e+01, 9.91394069e-02, 9.48666051e+00, 9.53121732e-01,
       5.07253124e+00, 8.31159974e+00, 5.98996443e-02, 3.63612996e+00,
       2.78768950e+00], index = X_test.columns)
col_drop = feature_importance[feature_importance<1] 
col_drop

In [ ]:
X_test = X_test[X_test.columns.drop(col_drop.index)]
X_train = X_train[X_train.columns.drop(col_drop.index)]

In [ ]:
num_features = ['P1','P3','P4']
cat_features = X_test.columns.drop(num_features).tolist()

# Transformer
numeric_transformer = StandardScaler()
cat_transformer = OneHotEncoder(drop='first', sparse=False)

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", cat_transformer, cat_features)
    ]
)

# Fit
preprocessor.fit(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
# Seed setting

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
print(device)

In [ ]:
# Input dim
input_shape = X_test.shape[1]
input_shape

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_shape):
        super(Classifier, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_shape,64),
            nn.LeakyReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 30),
            nn.LeakyReLU(),
            nn.BatchNorm1d(30),
            nn.Dropout1d(0.1),
            nn.Linear(30, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

# Initialize Net
model = Classifier(input_shape = input_shape)
print(model)

In [ ]:
# model load
checkpoint = torch.load('checkpoint.pt', map_location=device)
model.load_state_dict(checkpoint)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.df = X
        self.labels = y
    
    def __getitem__(self, index):
        self.x = self.df[index]
        self.y = self.labels[index]
        return torch.Tensor(self.x), self.y

    def __len__(self):
        return len(self.df)

In [ ]:
y_test = np.zeros(len(X_test))

In [ ]:
# To DataLoader
test = MyDataset(X_test, y_test)

test_loader = DataLoader(test, shuffle=False)

In [ ]:
# Predict

import itertools
from tqdm import tqdm

y_pred_list = []

# for prediction
model.eval()

with torch.no_grad():
    for data, target in tqdm(test_loader):
        data = data.to(device)
        y_test_pred = model(data)
        y_pred_tag = torch.round(y_test_pred).to('cpu')
        y_pred_list.append(y_pred_tag.detach().numpy())

y_pred_list = [i.squeeze().tolist() for i in y_pred_list]

In [ ]:
submission = pd.read_csv('과제2_예선_예측결과_김당찬.csv')
submission['winning'] = y_pred_list

In [ ]:
submission.to_csv('과제2_예선_예측결과_김당찬.csv', index=False)